In [1]:
import numpy as np

In [2]:
def normalize_and_add_ones(x):
    normalized = []
    restoration = {'max': x[:, -1].max(), 'min': x[:, -1].min()}
    for i in range(x.shape[1]):
        col = x[:, i]
        col = (col - col.min())/(col.max() - col.min())
        normalized.append(col)
    normalized = np.stack(normalized, axis=-1)
    one_col = np.ones([normalized.shape[0], 1])
    return (np.concatenate([one_col, normalized], axis=-1),
            restoration)

def load_dataset(path, drop_index=False, drop_header=False):
    with open(path) as file:
        data = file.readlines()
    for i in range(len(data)):
        data[i] = [float(i) for i in data[i].split()]
    data_tensor = np.array(data)
    if drop_index:
        data_tensor = data_tensor[:, 1:]
    if drop_header:
        data_tensor = data_tensor[1:, :]
    return data_tensor

In [25]:
class RidgeRegression:
    def __init__(self):
        self.weight = None
        self.best_LAMBDA = 0.

    def predict(self, x):
        return np.dot(x, self.weight)

    def train_step(self, x, y):
        assert len(x.shape) == 2 and x.shape[0] == y.shape[0]
        self.weight = np.linalg.inv(
            x.T.dot(x) + self.best_LAMBDA * np.identity(x.shape[1])
        ).dot(x.T).dot(y)

    def compute_RSS(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)

    def get_the_best_lambda(self, x, y):
        def cross_validation(num_folds):
            row_ids = np.arange(x.shape[0])
            valid_ids = np.split(
                row_ids[:len(row_ids) - len(row_ids) % num_folds], num_folds)
            valid_ids[-1] = np.append(valid_ids[-1],
                                      row_ids[len(row_ids) - len(row_ids)%num_folds:])
            train_ids = [[k for k in row_ids if k not in valid_ids[i]]
                         for i in range(num_folds)]
            aver_RSS = 0
            for i in range(num_folds):
                valid_part = {'X': x[valid_ids[i]], 'Y': y[valid_ids[i]]}
                train_part = {'X': x[train_ids[i]], 'Y': y[train_ids[i]]}
                self.train_step(train_part['X'], train_part['Y'])
                y_pred = self.predict(valid_part['X'])
                aver_RSS += self.compute_RSS(valid_part['Y'], y_pred)
            return aver_RSS / num_folds

        def range_scan(best_LAMBDA, minimum_RSS, LAMBDA_values):
            for current_LAMBDA in LAMBDA_values:
                aver_RSS = cross_validation(num_folds=5) 
                if aver_RSS < minimum_RSS:
                    best_LAMBDA = current_LAMBDA
                    minimum_RSS = aver_RSS
            return best_LAMBDA, minimum_RSS

        self.best_LAMBDA, minimum_RSS = range_scan(0, 1e8, range(50))
        LAMBDA_values = [k * 1./1000 for k in range(max(0, (self.best_LAMBDA - 1)*1000),
                                                    (self.best_LAMBDA + 1)*1000, 1)]
        self.best_LAMBDA, minimum_RSS = range_scan(self.best_LAMBDA,
                                                   minimum_RSS,
                                                   LAMBDA_values)

    def fit(self, x, y):
        self.get_the_best_lambda(x, y)
        self.train_step(x, y)

In [26]:
# load and preprocess the dataset
ds = load_dataset('dataset/deathrate.txt', drop_index=True)
ds, res = normalize_and_add_ones(ds)
x, y = ds[:, :-1], ds[:, -1]

# split the dataset
x_train, y_train = x[:50], y[:50]
x_test, y_test = x[50:], y[50:]

In [27]:
# create and train the model
model = RidgeRegression()
model.fit(x_train, y_train)

# show the best lambda and the loss
print('best lambda:', model.best_LAMBDA)
Y_predicted = model.predict(x_test)
print(model.compute_RSS(y_test, Y_predicted))

best lambda: 0
0.01524234785948798
